# Explore Data
- Then build book tree

In [52]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('goodreads_library_export.csv')

In [36]:
df.tail(2)

,Book Id,Title,Author,ISBN,ISBN13,My Rating,Average Rating,Publisher,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review
196,52036,Siddhartha,Hermann Hesse,"=""""","=""""",5,4.03,Bantam Books,152.0,1981.0,1922.0,NaN,2016/10/03,NaN,NaN,read,NaN
197,30659,Meditations,Marcus Aurelius,"=""0140449337""","=""9780140449334""",4,4.22,Penguin Books,303.0,2006.0,180.0,NaN,2016/10/03,NaN,NaN,read,NaN


In [5]:
print('Columns:', *df.columns, sep='\n- ')

Columns:
- Book Id
- Title
- Author
- Author l-f
- Additional Authors
- ISBN
- ISBN13
- My Rating
- Average Rating
- Publisher
- Binding
- Number of Pages
- Year Published
- Original Publication Year
- Date Read
- Date Added
- Bookshelves
- Bookshelves with positions
- Exclusive Shelf
- My Review
- Spoiler
- Private Notes
- Read Count
- Recommended For
- Recommended By
- Owned Copies
- Original Purchase Date
- Original Purchase Location
- Condition
- Condition Description
- BCID


In [38]:
df.columns

Index(['Book Id', 'Title', 'Author', 'Author l-f', 'Additional Authors',
       'ISBN', 'ISBN13', 'My Rating', 'Average Rating', 'Publisher', 'Binding',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Read', 'Date Added', 'Bookshelves', 'Bookshelves with positions',
       'Exclusive Shelf', 'My Review', 'Spoiler', 'Private Notes',
       'Read Count', 'Recommended For', 'Recommended By', 'Owned Copies',
       'Original Purchase Date', 'Original Purchase Location', 'Condition',
       'Condition Description', 'BCID'],
      dtype='object')

In [39]:
# Drop Columns
drop_columns = ['Book Id', 'Title', 'Author','My Rating', 'Average Rating',
       'Date Read', 'Date Added', 'Bookshelves',
       'Exclusive Shelf', 'My Review']

for col in df.columns:
    if col not in drop_columns:
        df.drop(col, inplace=True, axis=1)

In [40]:
# Get only read rows

df = df[df['Exclusive Shelf'] == 'read']

In [41]:
df.reset_index(inplace=True)

In [42]:
df.drop('index', inplace=True, axis=1)

/Users/natitaw/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
df['Date Read'] = pd.to_datetime(df['Date Read'])
df['Date Added'] = pd.to_datetime(df['Date Added'])

/Users/natitaw/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/natitaw/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
df.tail(70)

,Book Id,Title,Author,My Rating,Average Rating,Date Read,Date Added,Bookshelves,Exclusive Shelf,My Review
128,36391536,"She Has Her Mother's Laugh: The Powers, Perver...",Carl Zimmer,5,4.17,2019-03-24,2018-11-23,biology,read,This was a great book to read. Its full of fas...
129,35133922,Educated,Tara Westover,0,4.46,2019-03-02,2018-12-10,NaN,read,NaN
130,2657,To Kill a Mockingbird,Harper Lee,0,4.28,NaT,2019-02-27,NaN,read,NaN
131,2195464,What I Talk About When I Talk About Running,Haruki Murakami,5,3.85,2019-02-26,2019-02-14,NaN,read,This book is about the life of Haruki Murakami...
132,551520,The Wisdom of Insecurity: A Message for an Age...,Alan W. Watts,0,4.20,2019-02-15,2019-02-10,NaN,read,NaN
...,...,...,...,...,...,...,...,...,...,...
193,3228917,Outliers: The Story of Success,Malcolm Gladwell,4,4.16,NaT,2016-10-03,NaN,read,NaN
194,4069,Man's Search for Meaning,Viktor E. Frankl,5,4.36,NaT,2016-10-03,NaN,read,NaN
195,629,Zen and the Art of Motorcycle Maintenance: An ...,Robert M. Pirsig,4,3.77,NaT,2016-10-03,NaN,read,NaN
196,52036,Siddhartha,Hermann Hesse,5,4.03,NaT,2016-10-03,NaN,read,NaN


In [64]:
my_graph = dict()

for i in range(len(df)):
    
    book = df['Title'][i]
    my_graph[book] = []
    
    start = df['Date Read'][i]
    date_added = df['Date Added'][i]
    
    # Same family if its within two weeks
    for j in range(len(df)):
        end = df['Date Read'][j]
        book2 = df['Title'][i]
        print(start, end)
        if start != pd.NaT and end != pd.NaT and diff(start, end) < 2:
            my_graph[book].append(book2)

2020-11-24 00:00:00 2020-11-24 00:00:00
2020-11-24 00:00:00 2020-11-19 00:00:00
2020-11-24 00:00:00 2020-11-18 00:00:00
2020-11-24 00:00:00 2020-11-18 00:00:00
2020-11-24 00:00:00 2018-02-23 00:00:00
2020-11-24 00:00:00 2020-10-30 00:00:00
2020-11-24 00:00:00 NaT


ValueError: cannot convert float NaN to integer

In [49]:
df['Date Read'][33]

Timestamp('2020-04-16 00:00:00')

In [50]:
def diff(start, end):
    x = pd.to_datetime(end) - pd.to_datetime(start)
    return int(x / np.timedelta64(1, 'W'))

In [60]:
print(df['Date Read'][33], df['Date Read'][150])
diff(df['Date Read'][33], df['Date Read'][22])

2020-04-16 00:00:00 NaT


7

In [ ]:
graph = { "a" : ["c"],
          "b" : ["c", "e"],
          "c" : ["a", "b", "d", "e"],
          "d" : ["c"],
          "e" : ["c", "b"],
          "f" : []
        }

In [ ]:
def generate_edges(graph):
    edges = []
    for node in graph:
        for neighbour in graph[node]:
            edges.append((node, neighbour))

    return edges

print(generate_edges(graph))